In [1]:
!pip install mygene statannotations scrublet scanpy scvelo decoupler matplotlib_venn goatools gseapy scperturb biomart PyComplexHeatmap statsmodels omnipath git+https://github.com/saezlab/pypath.git --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyopenssl 23.0.0 requires cryptography<40,>=38.0.0, but you have cryptography 41.0.7 which is incompatible.


In [2]:
import subprocess
import os
import sys
import matplotlib.backends.backend_pdf
import scanpy as sc
import matplotlib.pyplot as pl
import anndata as ad
import pandas as pd
import numpy as np
import seaborn as sns

from pathlib import Path

# Jupyter stuff
from tqdm.notebook import tqdm
from IPython.display import clear_output
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

%matplotlib inline

# Custom functions
sys.path.insert(1, '../')
from utils import *

# scperturb package
sys.path.insert(1, '../package/src/')
from scperturb import *

from pathlib import Path
figure_path = Path('../figures/')

/tmp/ipykernel_89/1400239515.py:17: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [3]:
TEMPDIR = Path('/scratch/peidli/scPerturb/')

# Checks

In [14]:
adata = sc.read_h5ad(TEMPDIR / 'CuiHacohen2023.h5ad')

In [15]:
adata.obs['cytokine_family'].value_counts()

TNF                            18397
IL-6/IL-12                     13446
Other                          11070
Growth factor                  10416
Common beta chain/IL13/TSLP     9304
IL-1                            7907
Interferon                      6688
IL-17                           6409
IL-10                           5468
Common beta chain               3445
Complement                      2188
Control                         1296
Name: cytokine_family, dtype: int64

# New

In [4]:
celltypes = [
    'B_cell',
    'cDC1',
    'cDC2',
    'eTAC',
    'ILC',
    'Macrophage',
    'MigDC',
    'Monocyte',
    'Neutrophil',
    'NK_cell',
    'pDC',
    'T_cell_CD4',
    'T_cell_CD8',
    'T_cell_gd',
    'Treg'
]

In [19]:
adatas = []
for celltype in celltypes:
    print(celltype)
    adata = sc.read_h5ad(TEMPDIR / f'CuiHacohen2023/{celltype}_temp.h5ad')
    adatas.append(adata)

B_cell
cDC1
cDC2
eTAC
ILC
Macrophage
MigDC
Monocyte
Neutrophil
NK_cell
pDC
T_cell_CD4
T_cell_CD8
T_cell_gd
Treg


In [26]:
adata = sc.concat(adatas, axis=0)

In [29]:
adata.X

<96034x31053 sparse matrix of type '<class 'numpy.int64'>'
	with 148255987 stored elements in Compressed Sparse Row format>

In [24]:
# Obs
adata.obs.rename({
    'nCount_RNA': 'ncounts', 
    'nFeature_RNA': 'ngenes',
    'nCount_HTO': 'ncounts_tags',
    'biological_replicate_number': 'bio_replicate',
    'sample': 'perturbation',
    'rep': 'bio_replicate'
}, axis=1, inplace=True)
adata.obs.drop(['orig.ident'], axis=1, inplace=True)
adata.obs.perturbation = adata.obs.perturbation.astype(str)
adata.obs['perturbation'][adata.obs['perturbation']=='PBS'] = 'control'
adata.obs['nperts'] = [1-p.count('control') if type(p)==str else 0 for p in adata.obs.perturbation]
adata.obs['perturbation_type'] = 'cytokines'
adata.obs['disease'] = "healthy"
adata.obs['cancer'] = False
adata.obs['tissue_type']="primary"
adata.obs['organism'] = 'mouse'

/tmp/ipykernel_52/1894595031.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adata.obs['perturbation'][adata.obs['perturbation']=='PBS'] = 'control'


In [25]:
adata.obs

,ncounts,ngenes,channel,ncounts_tags,nFeature_HTO,MULTI_ID,MULTI_classification,percent_mito,RNA_snn_res.1.5,seurat_clusters,...,celltype,channel_hashtag,perturbation,bio_replicate,nperts,perturbation_type,disease,cancer,tissue_type,organism
CATAGACGTAACGATA-18,2081,910,ln-hash-cytokine18,2212,7,tag03,tag03-CTTGCCGCATGTCAT,5.478136,3,3,...,B_cell,ln-hash-cytokine18_tag03,41BBL,rep2,1,cytokines,healthy,False,primary,mouse
CGTGAATCAGCGAGTA-10,4082,1566,ln-hash-cytokine10,6308,7,tag05,tag05-CTTTGTCTTTGTGAG,5.609995,46,46,...,B_cell,ln-hash-cytokine10_tag05,41BBL,rep1,1,cytokines,healthy,False,primary,mouse
ACGGTCGGTAACCCTA-10,3084,1154,ln-hash-cytokine10,3115,7,tag05,tag05-CTTTGTCTTTGTGAG,4.701686,3,3,...,B_cell,ln-hash-cytokine10_tag05,41BBL,rep1,1,cytokines,healthy,False,primary,mouse
ACTTTGTGTACGAGTG-42,2947,1106,ln-hash-cytokine42,2192,7,tag01,tag01-ACCCACCAGTAAGAC,7.159824,3,3,...,B_cell,ln-hash-cytokine42_tag01,41BBL,rep3,1,cytokines,healthy,False,primary,mouse
GTGTAACCATGACTTG-18,1721,810,ln-hash-cytokine18,1955,7,tag03,tag03-CTTGCCGCATGTCAT,4.706566,3,3,...,B_cell,ln-hash-cytokine18_tag03,41BBL,rep2,1,cytokines,healthy,False,primary,mouse
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CATACTTGTAGTCACT-40,1348,648,ln-hash-cytokine40,845,6,tag04,tag04-AAAGCATTCTTCACG,3.560831,10,10,...,Treg,ln-hash-cytokine40_tag04,VEGF,rep3,1,cytokines,healthy,False,primary,mouse
GGAACCCAGACATAAC-40,5023,1690,ln-hash-cytokine40,2728,7,tag04,tag04-AAAGCATTCTTCACG,6.330878,10,10,...,Treg,ln-hash-cytokine40_tag04,VEGF,rep3,1,cytokines,healthy,False,primary,mouse
ATGGGAGGTAAGATTG-44,3359,1288,ln-hash-cytokine44,2123,7,tag04,tag04-AAAGCATTCTTCACG,7.025901,5,5,...,Treg,ln-hash-cytokine44_tag04,VEGF,rep2,1,cytokines,healthy,False,primary,mouse
ATGGATCCAGTCGGAA-40,3416,1441,ln-hash-cytokine40,4450,7,tag04,tag04-AAAGCATTCTTCACG,5.064403,5,5,...,Treg,ln-hash-cytokine40_tag04,VEGF,rep3,1,cytokines,healthy,False,primary,mouse


# OLD

In [4]:
ids = [f'0{x}' if x < 10 else str(x) for x in range(1, 46) if x != 12]  # sample 12 is missing

In [6]:
files = [TEMPDIR / "CuiHacohen2023/samples20_temp_annotated.h5ad", TEMPDIR / "CuiHacohen2023/samples21_temp_annotated.h5ad"]

In [7]:
adatas = []
for f in tqdm(files):
    adata = sc.read(f)
    adatas.append(adata)

  0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


In [17]:
adata = sc.concat(adatas, axis=0)

In [18]:
# Obs
adata.obs.rename({
    'nCount_RNA': 'ncounts', 
    'nFeature_RNA': 'ngenes',
    'nCount_HTO': 'ncounts_tags',
    'filename_prefix': 'sample',
    'processing_batch': 'batch',
    'biological_replicate_number': 'bio_replicate',
    'cytokine': 'perturbation'
}, axis=1, inplace=True)
adata.obs.drop(['nFeature_HTO'], axis=1, inplace=True)
adata.obs.perturbation = adata.obs.perturbation.astype(str)
adata.obs['perturbation'][pd.isna(adata.obs['perturbation'])] = 'control'
adata.obs = adata.obs[['perturbation', 'batch', 'bio_replicate', 'sample', 'ncounts', 'ngenes', 'ncounts_tags', 'hashtag_ID']]
adata.obs['nperts'] = [1-p.count('control') if type(p)==str else 0 for p in adata.obs.perturbation]
adata.obs['perturbation_type'] = 'cytokines'
adata.obs['disease'] = "healthy"
adata.obs['cancer'] = False
adata.obs['tissue_type']="primary"
adata.obs["celltype"] = 'mixed cells from draining lymph nodes'
adata.obs['organism'] = 'mouse'

/tmp/ipykernel_50/606385222.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adata.obs['perturbation'][pd.isna(adata.obs['perturbation'])] = 'control'


In [19]:
adata.obs

,perturbation,batch,bio_replicate,sample,ncounts,ngenes,ncounts_tags,hashtag_ID,nperts,perturbation_type,disease,cancer,tissue_type,celltype,organism
AAACCCAAGCCAGACA-20,PBS,7.0,rep8,cytokine-samples20,125,110,300,7,1,cytokines,healthy,False,primary,mixed cells from draining lymph nodes,mouse
AAACCCAAGCCTCAAT-20,IL-17A,7.0,rep1,cytokine-samples20,245,194,388,1,1,cytokines,healthy,False,primary,mixed cells from draining lymph nodes,mouse
AAACCCAAGTTTCAGC-20,IL-17C,7.0,rep1,cytokine-samples20,103,93,683,3,1,cytokines,healthy,False,primary,mixed cells from draining lymph nodes,mouse
AAACCCACACCAAATC-20,PBS,7.0,rep8,cytokine-samples20,144,119,229,7,1,cytokines,healthy,False,primary,mixed cells from draining lymph nodes,mouse
AAACCCAGTACTAACC-20,PBS,7.0,rep8,cytokine-samples20,134,101,232,7,1,cytokines,healthy,False,primary,mixed cells from draining lymph nodes,mouse
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTTGAGGGCCTCT-21,PBS,7.0,rep7,cytokine-samples21,105,95,408,6,1,cytokines,healthy,False,primary,mixed cells from draining lymph nodes,mouse
TTTGTTGCAATCCTTT-21,C3a,7.0,rep2,cytokine-samples21,505,306,360,1,1,cytokines,healthy,False,primary,mixed cells from draining lymph nodes,mouse
TTTGTTGCAATTCGTG-21,C3a,7.0,rep2,cytokine-samples21,114,96,283,1,1,cytokines,healthy,False,primary,mixed cells from draining lymph nodes,mouse
TTTGTTGCAGAGGCTA-21,nan,7.0,NaN,cytokine-samples21,156,94,635,5,0,cytokines,healthy,False,primary,mixed cells from draining lymph nodes,mouse


# Checking

In [4]:
adata = sc.read(TEMPDIR / "CuiHacohen2023.h5ad")

/opt/conda/lib/python3.10/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


In [11]:
adata.obs

,perturbation,batch,bio_replicate,sample,ncounts,ngenes,ncounts_tags,hashtag_ID,nperts,perturbation_type,disease,cancer,tissue_type,celltype,organism,percent_mito,percent_ribo
cell_barcode,,,,,,,,,,,,,,,,,
AAACCCAAGGTCCTGC-01,IL-3,1.0,rep1,cytokine-samples01,163,145,872,4,1,cytokines,healthy,False,primary,mixed cells from draining lymph nodes,mouse,1.226994,30.674847
AAACCCAAGTGGACGT-01,PBS,1.0,rep1,cytokine-samples01,125,109,469,8,1,cytokines,healthy,False,primary,mixed cells from draining lymph nodes,mouse,9.600000,28.000000
AAACCCACAACCGGAA-01,PBS,1.0,rep1,cytokine-samples01,132,113,200,8,1,cytokines,healthy,False,primary,mixed cells from draining lymph nodes,mouse,0.757576,27.272727
AAACCCACAATTTCTC-01,PBS,1.0,rep1,cytokine-samples01,162,137,211,8,1,cytokines,healthy,False,primary,mixed cells from draining lymph nodes,mouse,0.617284,23.456790
AAACCCAGTCGGCTAC-01,IL-3,1.0,rep1,cytokine-samples01,250,121,633,4,1,cytokines,healthy,False,primary,mixed cells from draining lymph nodes,mouse,49.200000,14.800000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTTGGTCAACACT-45,PBS,13.0,rep14,cytokine-samples45,296,213,149,7,1,cytokines,healthy,False,primary,mixed cells from draining lymph nodes,mouse,3.378378,39.527027
TTTGTTGTCAGAGCAG-45,PBS,13.0,rep14,cytokine-samples45,285,188,125,7,1,cytokines,healthy,False,primary,mixed cells from draining lymph nodes,mouse,0.701754,45.263158
TTTGTTGTCAGCAATC-45,PBS,13.0,rep14,cytokine-samples45,2,2,11,7,1,cytokines,healthy,False,primary,mixed cells from draining lymph nodes,mouse,0.000000,0.000000


In [10]:
adata.obs.perturbation.value_counts()

PBS       349669
RANKL      31405
IL-36α     29106
41BBL      23086
IL-1Ra     14005
           ...  
IL-17B       349
IL-20        289
IL-23        207
IL-17E       207
HGF          202
Name: perturbation, Length: 88, dtype: int64